<a href="https://colab.research.google.com/github/ruchikaverma-iitg/ML-DL-RL_Codes/blob/master/Machine_Learning/Murphy/COVID-19_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.3.1 Example: Bayes rule for COVID-19 diagnosis

Consider estimating if someone has COVID $H=1$ or not $H=0$ on the basis of a PCR test. The test can either return a positive result $Y=1$ or a negative result $Y=0$. The reliability of the test is given by the following observation model.

p(Y = 1 | H = 1) = True Positive Rate (TPR) = Sensitivity

P(Y = 0 | H = 0) = True Negative Rate (TNR) = Specificity

P(Y = 1 | H = 0) = False Positive Rate (FPR) = 1-TNR

P(Y = 0 | H = 1) = False Negative Rate (FNR) = 1-TPR

Using data from https://www.nytimes.com/2020/08/04/science/coronavirus-bayes-statistics-math.html, we set sensitivity to 87.5% and the specificity to 97.5%.

We also need to specify the prior probability  𝑝(𝐻=1) ; this is known as the prevalence. This varies over time and place, but let's pick  𝑝(𝐻=1)=0.1  as a reasonable estimate.

If you test positive:

\begin{align}
p(H=1|Y=1) 
 &= \frac{p(Y=1|H=1) p(H=1)}
{p(Y=1|H=1) p(H=1) + p(Y=1|H=0) p(H=0)}
= \frac{TPR * prior}
{TPR* prior + FPR* (1-prior)}
= 0.795
\end{align}

If you test negative:
\begin{align}
p(H=1|Y=0) 
 &= \frac{p(Y=0|H=1) p(H=1)}
{p(Y=0|H=1) p(H=1) + p(Y=0|H=0) p(H=0)}
= \frac{FNR * prior}
{FNR * prior + TNR*(1-prior)}
=0.014
\end{align}

In [15]:
import numpy as np

def normalize(x):
  return x / np.sum(x)

def posterior_covid(observed, prevalence=None, sensitivity=None):
  # Y (test result): observed = 0 for negative test, 1 for positive test
  # H (actual state - infected/not infected): hidden state = 0 if no-covid, 1 if have-covid

  if sensitivity is None:
    sensitivity = 0.875

  specificity = 0.975

  TPR = sensitivity; 
  FNR = 1-TPR
  TNR = specificity
  FPR = 1-TNR

  # prior(hidden)
  if prevalence is None:
    prevalence = 0.1

  # likelihood(hidden, obs)
  likelihood_fn = np.array([[TNR, FPR], [FNR, TPR]])# 2x2 matrix where test results are on x - axis and ground-truth on y-axis

  prior = np.array([1-prevalence, prevalence])
  likelihood = likelihood_fn[:, observed].T
  posterior = normalize(prior * likelihood)
  return posterior

For a prevalence of  p(H=1)=0.1

In [16]:
print(f"Probability of being infected (H=1) given the test is positive (Y=1), {posterior_covid(observed=1)[1]*100}")
print(f"Probability of being infected (H=1) given the test is negative (Y=0), {posterior_covid(observed=0)[1]*100}")

Probability of being infected (H=1) given the test is positive (Y=1), 79.54545454545453
Probability of being infected (H=1) given the test is negative (Y=0), 1.4044943820224722


For a prevalence of  p(H=1)=0.01

In [17]:
print(f"Probability of being infected (H=1) given the test is positive (Y=1),{posterior_covid(observed = 1, prevalence = 0.01)[1]*100}") # positive test 
print(f"Probability of being infected (H=1) given the test is negative (Y=0), {posterior_covid(observed = 0, prevalence=0.01)[1]*100}") # negative test

Probability of being infected (H=1) given the test is positive (Y=1),26.119402985074615
Probability of being infected (H=1) given the test is negative (Y=0), 0.12933264355923438


In [26]:
pop = 100000 #population
infected = 0.01*pop #Total positives (H=1)
print(f"Infected ", infected)
sens = 87.5/100
spec = 97.5/100
FPR = 1-spec
FNR = 1-sens
print(f"FPR and FNR - {[FPR, FNR]}")

true_pos = sens * infected
false_pos = FPR * (pop-infected)

print(f"TP and FP - {[true_pos, false_pos]}")

num_pos = true_pos + false_pos
posterior = true_pos/num_pos
print(f"#Positives and Posterior-{[num_pos, posterior]}")

Infected  1000.0
FPR and FNR - [0.025000000000000022, 0.125]
TP and FP - [875.0, 2475.0000000000023]
#Positives and Posterior-[3350.0000000000023, 0.2611940298507461]
